# Логистическая регрессия

Практическая лекция по логистической регрессии.

На этом занятии мы будем работать с тем же набором данных, что и в занятии про Knn. Так что посмотреть, что мы делали с данными, можнно там.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
data = pd.read_csv('data/Pokemon.csv', index_col=0).reset_index(drop=True)

In [ ]:
data.head()

In [ ]:
num_cols = [col for col in data.columns if data[col].dtype == 'float64' or data[col].dtype == 'int64']

In [ ]:
data.loc[data['Type 2'].isnull(), 'Type 2'] = 'NoneType'

## Преобразование категориальных признаков 

Линейным моделям не страшна размерность пространства признаков, так что для категориальных добавим дамми переменные. Но LabelEncoding тоже сохраним.

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
data['Type 1'] = encoder.fit_transform(data['Type 1'])
data['Type 2'] = encoder.fit_transform(data['Type 2'])

In [ ]:
type1 = pd.get_dummies(data['Type 1'], drop_first=True)
type1.columns = ['type1_'+str(i) for i in range(type1.shape[1])]
type2 = pd.get_dummies(data['Type 2'], drop_first=True)
type2.columns = ['type2_'+str(i) for i in range(type2.shape[1])]


In [ ]:
data.drop('Name', axis=1, inplace=True)

In [ ]:
data = pd.concat([data, type1, type2], axis=1)

Добавим все двойные кросс признаки. Константный признак включать не будем.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

pol_feat = PolynomialFeatures(degree=2, include_bias=False)
pol = pd.DataFrame(pol_feat.fit_transform(data.drop('Legendary', axis=1))).reset_index(drop=True)
pol.columns = ['pol_'+str(i) for i in range(pol.shape[1])]

In [ ]:
pol.head()

In [ ]:
data = pd.concat([data, pol], axis=1)

In [ ]:
data.head()

# Построение модели логистической регрессии

## Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data.drop('Legendary', axis=1), 
                                                    data['Legendary'], test_size=0.3, 
                                                    random_state=42)

## Training and Predicting

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logmodel = LogisticRegression(C=2, max_iter=1000, warm_start=True, random_state=42)
logmodel.fit(X_train, y_train)

In [ ]:
predictions = logmodel.predict_proba(X_test)[:, 1]

## Оценка качества модели

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
print(roc_auc_score(y_test, predictions))

## Cross Validation

In [ ]:
target = data['Legendary']*1

In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=10, shuffle=True, random_state=42)
val_rate = []
for tr_ind, val_ind in kf.split(data):
    lr = LogisticRegression(C=2, max_iter=1000, warm_start=True, random_state=42)
    train = data.drop('Legendary', axis=1).loc[tr_ind]
    val = data.drop('Legendary', axis=1).loc[val_ind]

    target_train = target[tr_ind]
    target_val = target[val_ind]

    lr.fit(train, target_train)
    pred_i = lr.predict_proba(val)[:, 1]
    val_rate.append(roc_auc_score(target_val, pred_i))

In [ ]:
print('Среднее: {:.3f}\nCтандартное отклонение: {:.3f}'.format(np.mean(val_rate), np.std(val_rate)))

# Домашнее задание

Сделать модель с лучшим скором. 

Параметры кросс-валидации зафиксированы, в остальном полная свобода действий.
> kf = KFold(n_splits=10, shuffle=True, random_state=42)

Советы:
* Отбор признаков в данном случае очень важен. Мой бейзлайн на 0.958 на небольшом кол-ве признаков. Его надо побить;
* Например, можно отбирать с помощью значимости по p-value. Как это сделать в Python можно посмотреть тут: https://stackoverflow.com/questions/27928275/find-p-value-significance-in-scikit-learn-linearregression;
* Попробуйте скалировать признаки, например, через StandardScaler или через MinMaxScaler.


## Отличная работа!

Не сдавайтесь!